# Black & Litterman Factor Model

We build a Maximum Sharpe Ratio portfolio using Factor Model for estimation, and incorporate views on factors using Black-Litterman.

## Data

In [1]:
from plotly.io import show
from sklearn.model_selection import train_test_split

from skfolio import Population, RiskMeasure
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from skfolio.optimization import MeanRisk, ObjectiveFunction
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import BlackLitterman, FactorModel

prices = load_sp500_dataset()
factor_prices = load_factors_dataset()

prices = prices["2014":]
factor_prices = factor_prices["2014":]

X, y = prices_to_returns(prices, factor_prices)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

In [2]:
X_train.head()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
2014-01-03,-0.021941,0.012658,0.019298,0.004423,0.001694,-0.000724,-0.001591,0.009005,0.007727,-0.004921,0.007290,0.004831,-0.006747,0.001696,0.001976,-0.001119,-0.014801,0.007104,-0.003300,-0.002409
2014-01-06,0.005417,0.032500,0.015233,-0.031223,-0.002663,-0.008007,-0.009639,0.005223,0.005801,-0.004678,0.008424,0.000198,-0.021116,0.000495,0.000986,0.002372,0.006461,-0.011444,-0.005591,0.001503
2014-01-07,-0.007145,0.012107,-0.009646,-0.026135,0.008466,0.001096,0.004920,0.021230,-0.011535,0.002954,-0.006601,0.007464,0.007758,0.014579,0.006206,0.009661,0.017028,0.030569,0.003078,0.014147
2014-01-08,0.006311,0.000000,0.004834,-0.014072,-0.014226,-0.002926,0.005277,-0.001371,0.009434,-0.011147,-0.001571,-0.006399,-0.017865,-0.002880,0.006853,-0.014483,0.001090,-0.011626,-0.007907,-0.003259
2014-01-09,-0.012719,-0.021531,0.015078,-0.008176,0.000000,0.000366,-0.004385,0.006056,-0.001860,-0.005247,0.011157,-0.005423,-0.006449,-0.004688,-0.000681,0.002246,-0.015635,0.006088,0.003346,-0.009735


In [3]:
y_train.head()

,MTUM,QUAL,SIZE,USMV,VLUE
Date,,,,,
2014-01-03,0.001670,-0.001965,-0.005389,-0.000273,-0.001169
2014-01-06,-0.002178,-0.003917,0.000000,-0.002284,-0.000170
2014-01-07,0.008258,0.008072,0.000185,0.005707,0.005576
2014-01-08,0.007343,-0.000371,0.000000,-0.000272,0.000000
2014-01-09,0.001832,-0.000537,-0.002093,0.001428,-0.002180


## Prior View

Views about the future realization of factors (convert to daily estimates).

In [4]:
factor_views = [
    "SIZE == 0.00039",
    "SIZE - VLUE == 0.00011 ",
    "MTUM - QUAL == 0.00007",
]

## Black-Litterman Factor Model

In [ ]:
max_sharpe_model_bl_factor = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    prior_estimator=FactorModel(
        factor_prior_estimator=BlackLitterman(
            views=factor_views
        )
    ),
    portfolio_params=dict(name="Black-Litterman Factor Model")
)

max_sharpe_model_bl_factor.fit(X_train, y_train)
max_sharpe_model_bl_factor.weights_

array([5.55998971e-02, 2.42678756e-02, 3.63991372e-02, 1.97989097e-02,
       4.38883298e-08, 1.69582846e-02, 1.34103109e-01, 3.52966988e-07,
       2.78013243e-02, 9.56133325e-02, 6.72490988e-02, 9.04221682e-02,
       1.24559355e-01, 8.98176230e-02, 5.85674115e-02, 2.83525179e-02,
       6.69595123e-03, 1.02316969e-01, 2.14765919e-02, 4.61287983e-08])

Create a vanilla Maximum Sharpe Ratio model.

In [7]:
max_sharpe_model_factor = MeanRisk(
    risk_measure=RiskMeasure.VARIANCE,
    objective_function=ObjectiveFunction.MAXIMIZE_RATIO,
    prior_estimator=FactorModel(),
    portfolio_params=dict(name="Factor Model")
)

max_sharpe_model_factor.fit(X_train, y_train)
max_sharpe_model_factor.weights_

array([1.03294289e-06, 1.27482685e-03, 4.19682803e-07, 3.34130825e-06,
       7.36838286e-07, 1.28824408e-06, 5.13031432e-02, 6.35619183e-02,
       6.14804833e-07, 1.79106051e-01, 5.03130911e-02, 7.13734379e-02,
       4.13002526e-02, 2.27978407e-01, 5.13348034e-02, 1.44130375e-01,
       2.99026116e-07, 6.19737850e-02, 5.63413085e-02, 8.67773196e-07])

## Prediction

In [10]:
pred_bl_factor = max_sharpe_model_bl_factor.predict(X_test)
pred_factor = max_sharpe_model_factor.predict(X_test)

## Analysis

In [11]:
population = Population([pred_bl_factor, pred_factor])
population.plot_cumulative_returns()

In [12]:
population.plot_composition()

## A Bit Further

Views on both assets and factors.

In [14]:
assets_views = [
    "AAPL == 0.00098",
    "AAPL - GE == 0.00086",
    "JPM - GE == 0.00059",
]

In [17]:
bl_model_factor_bl = BlackLitterman(
    views=assets_views,
    prior_estimator=FactorModel(
        factor_prior_estimator=BlackLitterman(
            views=factor_views
        )
    )
)

bl_model_factor_bl.fit(X, y)

BlackLitterman(prior_estimator=FactorModel(factor_prior_estimator=BlackLitterman(views=['SIZE '
                                                                                        '== '
                                                                                        '0.00039',
                                                                                        'SIZE '
                                                                                        '- '
                                                                                        'VLUE '
                                                                                        '== '
                                                                                        '0.00011 ',
                                                                                        'MTUM '
                                                                                        '- '
                                                                                        'QUAL '
                                                                                        '== '
                                                                                        '0.00007'])),
               views=['AAPL == 0.00098', 'AAPL - GE == 0.00086',
                      'JPM - GE == 0.00059'])

In [21]:
bl_model_factor_bl.prior_model_.mu

array([6.68345051e-04, 6.53934834e-04, 3.90848082e-04, 3.98903200e-04,
       3.26667558e-04, 4.81149412e-05, 3.80152289e-04, 2.16630514e-04,
       4.15987823e-04, 2.22228126e-04, 2.65981416e-04, 2.31726382e-04,
       4.89068993e-04, 2.49888614e-04, 2.14239879e-04, 2.07643522e-04,
       3.26206915e-04, 3.39434842e-04, 1.92442967e-04, 2.81071223e-04])

In [27]:
bl_model_factor_bl.prior_model_.covariance.shape

(20, 20)